# Movie Recommendation System

In [126]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-09-06 17:17:08--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  1.41MB/s    in 1m 54s  

2020-09-06 17:19:05 (1.34 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [362]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read the **movies.csv** file into their Dataframes:


In [363]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Let's also remove the year from the **title** column by using pandas' replace function and store in a new **year** column.


In [364]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the **Genres** column into a **list of Genres** to simplify future use. This can be achieved by applying Python's split string function on the correct column.


In [365]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [366]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.


In [367]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let us take input from user : 

Notice: To add more movies, simply increase the amount of elements in the **userInput**. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [429]:
userInput = [
                {'title':'Darna Mana Hai', 'rating':4.7},
                 {'title':'Insidious: Chapter 2', 'rating':5.0},
                 {'title':'Insidious Chapter 3', 'rating':4.5},
                 {'title':'Insidious', 'rating':5}
#                 {'title':'Bhool Bhulaiyaa', 'rating':5.0}
         ] 

inputMovies = pd.DataFrame(userInput)
ind = len(inputMovies.index)
inputMovies

,rating,title
0,4.7,Darna Mana Hai
1,5.0,Insidious: Chapter 2
2,4.5,Insidious Chapter 3
3,5.0,Insidious


#### Add movieId to input user

With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

In [430]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,85788,Insidious,5.0
1,104908,Insidious: Chapter 2,5.0
2,122884,Insidious Chapter 3,4.5
3,135593,Darna Mana Hai,4.7


#### The users who have seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [431]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
6908,63,85788,2.5
48582,551,85788,2.5
69208,768,85788,3.5
78030,815,85788,2.5
110069,1182,85788,3.5


We now group up the rows by user ID.


In [432]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [433]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(38949,          userId  movieId  rating
  3599868   38949    85788     4.0
  3600027   38949   104908     2.5
  3600132   38949   122884     2.0
  3600201   38949   135593     2.5), (2469,         userId  movieId  rating
  226175    2469    85788     4.0
  226177    2469   104908     5.0
  226178    2469   122884     3.0), (4280,         userId  movieId  rating
  397790    4280    85788     4.0
  397901    4280   104908     3.0
  397931    4280   122884     3.5)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.


In [434]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [435]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

pearsonCorrelationDict.items()

[(33280, 0.5000000000000107),
 (30721, 0),
 (162305, -0.5000000000000107),
 (24585, 0),
 (5133, 0),
 (187923, 0.9999999999999858),
 (19998, 1.0),
 (47648, 0.4999999999999867),
 (221732, 0.9999999999999787),
 (1573, 1.0),
 (61991, 0),
 (107061, 0.9878291611472452),
 (177719, -0.4999999999999867),
 (34381, 0.9999999999999827),
 (169555, 0.5000000000000053),
 (53338, 0),
 (11357, 0),
 (191073, 0),
 (40409, 0.9176629354822312),
 (81514, 0.8660254037844264),
 (19569, 0),
 (169074, 0.9999999999999787),
 (139382, 0.999999999999968),
 (25449, 0),
 (204421, 0.9999999999999827),
 (130183, 0),
 (21128, -1.0),
 (180362, 0),
 (27277, 0),
 (132751, 0.9999999999999787),
 (20640, 0),
 (242340, 0.9999999999999947),
 (15017, 0),
 (82604, 0),
 (75952, 0.5000000000000053),
 (110258, 0.5000000000000053),
 (167603, 0.9819805060619519),
 (4280, 0.0),
 (146623, 0.8660254037844264),
 (21696, 0),
 (25120, 0),
 (135877, 0.9707253433941353),
 (45259, 0),
 (193229, 0.4999999999999867),
 (16084, 0),
 (64215, 0),
 (

In [436]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.5,33280
1,0.0,30721
2,-0.5,162305
3,0.0,24585
4,0.0,5133


#### The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.

In [437]:
#print(topUsers.shape)
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()
#print(topUsers.shape)

,similarityIndex,userId
6,1.0,19998
9,1.0,1573
56,1.0,205117
31,1.0,242340
68,1.0,244588


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.

In [438]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,19998,110,3.0
1,1.0,19998,337,5.0
2,1.0,19998,356,5.0
3,1.0,19998,593,5.0
4,1.0,19998,1136,3.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [439]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,19998,110,3.0,3.0
1,1.0,19998,337,5.0,5.0
2,1.0,19998,356,5.0,5.0
3,1.0,19998,593,5.0,5.0
4,1.0,19998,1136,3.5,3.5


In [440]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,30.702758,116.083573
2,23.480246,75.164927
3,4.833299,12.859780
4,0.944911,2.362278
5,4.700840,15.258029


In [441]:
#Creates an empty dataframe
recommendation_df_collab = pd.DataFrame()
#Now we take the weighted average
recommendation_df_collab['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df_collab['movieId'] = tempTopUsersRating.index
recommendation_df_collab.head()

,weighted average recommendation score,movieId
movieId,,
1,3.780884,1
2,3.201198,2
3,2.660663,3
4,2.500000,4
5,3.245809,5


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [442]:
recommendation_df_collab = recommendation_df_collab.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df_collab['weighted average recommendation score']
recommendation_df_collab.head()

,weighted average recommendation score,movieId
movieId,,
135456,5.0,135456
3224,5.0,3224
92210,5.0,92210
112807,5.0,112807
6772,5.0,6772


In [443]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df_collab.head(20)['movieId'].tolist())]

,movieId,title,genres,year
307,310,Rent-a-Kid,[Comedy],1995
3138,3224,Woman in the Dunes (Suna no onna),[Drama],1964
3770,3862,About Adam,[Comedy],2000
4819,4914,Breathless (À bout de souffle),"[Crime, Drama, Romance]",1960
6663,6772,To Be and to Have (Être et avoir),[Documentary],2002
7748,8327,Dolls,"[Drama, Romance]",2002
8536,25990,It Should Happen to You,"[Comedy, Romance]",1954
9961,32657,"Man Who Planted Trees, The (Homme qui plantait...","[Animation, Drama]",1987
13022,61967,"Autumn Afternoon, An (Sanma no aji)",[Drama],1962
13786,68945,Neon Genesis Evangelion: Death & Rebirth (Shin...,"[Action, Animation, Mystery, Sci-Fi]",1997


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [444]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
16983,85788,Insidious,"[Fantasy, Horror, Thriller]",2010,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21752,104908,Insidious: Chapter 2,"[Horror, Thriller]",2013,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26469,122884,Insidious Chapter 3,[Thriller],2015,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29701,135593,Darna Mana Hai,[Horror],2003,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [445]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.


In [446]:
inputMovies['rating']

0    5.0
1    5.0
2    4.5
3    4.7
Name: rating, dtype: float64

In [447]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure              0.0
Animation              0.0
Children               0.0
Comedy                 0.0
Fantasy                5.0
Romance                0.0
Drama                  0.0
Action                 0.0
Crime                  0.0
Thriller              14.5
Horror                14.7
Mystery                0.0
Sci-Fi                 0.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [448]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [449]:
genreTable.shape

(34208, 20)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [450]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.146199
2    0.146199
3    0.000000
4    0.000000
5    0.000000
dtype: float64

In [451]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
ind2 = len(recommendationTable_df.index)
#Just a peek at the values
recommendationTable_df.head()

movieId
3837      1.0
1345      1.0
128952    1.0
53046     1.0
2617      1.0
dtype: float64

Now here's the Final recommendation table!


In [454]:
#num is the Number of recommendations according to your content from which you want to get top num_of_movies movies which users similar to you liked
num = ind2/ind
if ind2/ind < 3000:
    num = 3000
#num_of_movies is the Number of recommendations you want in the final table
num_of_movies = 20
Final = pd.DataFrame()
Final = movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(num).keys())]


y = []
mydefault = 0;
try:
    recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[0]]
except KeyError:
    x = mydefault
for i in range(0,num):
    x = 1
    try:
        recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[i]]
    except KeyError:
        x = mydefault
    if x==1:
        y.append(recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[i]])
    else:
        y.append(-1)

Final.insert(loc = 2,column = 'predicted rating',value = y)
Final_result = Final.sort_values(by='predicted rating',ascending=False)
Final_result.head(num_of_movies)



,movieId,title,predicted rating,genres,year
30229,137022,A Nanny's Revenge,5.000000,"[Drama, Thriller]",2012
30611,138684,Waar,5.000000,"[Action, Drama, Thriller]",2013
28630,132120,A House In The Hills,5.000000,"[Romance, Thriller]",1993
23900,112854,"Ghost Goes West, The",5.000000,"[Comedy, Fantasy, Horror, Romance]",1935
30284,137290,Rock Slyde,5.000000,"[Comedy, Mystery, Thriller]",2010
32432,144604,Running Out of Time 2,5.000000,"[Action, Crime, Thriller]",2001
32433,144606,Confessions of a Dangerous Mind,5.000000,"[Comedy, Crime, Drama, Romance, Thriller]",2002
21620,104441,"Frozen Ground, The",5.000000,"[Crime, Drama, Thriller]",2013
6618,6727,Targets,5.000000,"[Crime, Thriller]",1968
1902,1985,Halloween 4: The Return of Michael Myers,5.000000,[Horror],1988
